The Trajectory Rotation Average $ (\mathrm{\overline{TRA}}_{t_0}^{t_N}) $ is computed from the velocity along the trajectories launched from initial conditions distributed over a regular meshgrid of size (Ny, Nx). The time-interval $ [t_0, t_N] $ is discretized with a resolution of N.

\begin{equation}
\mathrm{\overline{TRA}}_{t_0}^{t_N} = \dfrac{1}{t_N-t_0} \sum_{i = 0}^{N-1} \cos^{-1}\dfrac{\langle \mathbf{\dot{x}}(t_i), \mathbf{\dot{x}}(t_{i+1}) \rangle+\mathrm{v}_0^2}{\sqrt{|\mathbf{\dot{x}}(t_i)|^2+\mathrm{v}_0^2}\sqrt{|\mathbf{\dot{x}}(t_{i+1})|^2+\mathrm{v}_0^2}}
\end{equation}

| Name | Type (Shape) | Description |
| --- | --- | --- |
| lenT | float | $ t_N-t_0 $ |
| v0 | float | $ \mathrm{v}_0 $|
| velocity_grid | array (Ny, Nx, Nz, N, 3) | velocity along trajectories launched from <br /> meshgrid of size (Ny, Nx, Nz) over the <br /> discretized time-interval with resolution N|
| TRA | array (Ny, Nx, Nz) | $ \mathrm{\overline{TRA}}_{t_0}^{t_N} $|

In [1]:
import numpy as np
from numba import njit, prange

@njit(parallel=True)
def _TRA(lenT, velocity_grid, v0 = 0):
        
    TR = np.zeros((velocity_grid.shape[0], velocity_grid.shape[1], velocity_grid.shape[2], velocity_grid.shape[3]-1))
        
    for t in prange(velocity_grid.shape[3]-1):
            
        for i in prange(velocity_grid.shape[0]):
            
            for j in prange(velocity_grid.shape[1]):
                
                for k in prange(velocity_grid.shape[2]):
                    
                    velx0 = velocity_grid[i, j, k, t, 0]
                    vely0 = velocity_grid[i, j, k, t, 1]
                    velz0 = velocity_grid[i, j, k, t, 2]
                    
                    vel0 = np.sqrt(velx0**2+vely0**2+velz0**2+v0**2)
                    
                    velx1 = velocity_grid[i, j, k, t + 1, 0]
                    vely1 = velocity_grid[i, j, k, t + 1, 1]
                    velz1 = velocity_grid[i, j, k, t + 1, 2]
                
                    vel1 = np.sqrt(velx1**2+vely1**2+velz1**2+v0**2)
                
                    TR[i, j, k, t] = np.abs(np.arccos((velx0*velx1+vely0*vely1+velz0*velz1+v0**2)/(vel0*vel1)))
        
    TRA = np.sum(TR, axis = 3)/lenT
        
    return TRA